## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

In [2]:
# Import and read the charity_data.csv
raw_data_df = pd.read_csv('resources/charity_data.csv')
raw_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
alphabet_df = raw_data_df.drop(['EIN', 'NAME'], axis=1)
alphabet_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column
print(alphabet_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
alphabet_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cut_off = 500

application_types_to_replace = alphabet_df['APPLICATION_TYPE'].value_counts()[alphabet_df['APPLICATION_TYPE'].value_counts() < cut_off].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    alphabet_df['APPLICATION_TYPE'] = alphabet_df['APPLICATION_TYPE'].replace(app, 'Other')

# Check to make sure binning was successful
alphabet_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
alphabet_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2170        1
C1820        1
C2561        1
C4200        1
C1370        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
alphabet_df['CLASSIFICATION'].value_counts()[alphabet_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
C0           3
C1260        3
C2710        3
C3200        2
C1256        2
C1246        2
C1234        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cut_off = 500

classifications_to_replace = alphabet_df['CLASSIFICATION'].value_counts()[alphabet_df['CLASSIFICATION'].value_counts() < cut_off].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    alphabet_df['CLASSIFICATION'] = alphabet_df['CLASSIFICATION'].replace(cls, 'Other')
    
# Check to make sure binning was successful
alphabet_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Out of curiosity, how many applications have a status of zero?
alphabet_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [11]:
# Discard all applications with status zero
alphabet_df = alphabet_df[alphabet_df['STATUS'] == 1]
alphabet_df.shape

(34294, 10)

In [12]:
# Determine which data points are outside of the 1.5*IQR range
ask_amt = alphabet_df['ASK_AMT']

quartiles = np.quantile(ask_amt,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)

# Drop the outliers
potential_outliers = (ask_amt <= upper_bound) | (ask_amt >= lower_bound)
alphabet_df = alphabet_df.loc[potential_outliers,:]
alphabet_df.shape

(34294, 10)

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df = pd.get_dummies(alphabet_df)
app_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
y = app_dummies_df['IS_SUCCESSFUL'].values
X = app_dummies_df.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## First Attempt

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]

nn_units = nn_input_dim*2

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units, activation='relu', input_dim=nn_input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/4, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 88)                3960      
_________________________________________________________________
dense_1 (Dense)              (None, 44)                3916      
_________________________________________________________________
dense_2 (Dense)              (None, 22)                990       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 23        
Total params: 8,889
Trainable params: 8,889
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=25)

Train on 25720 samples
Epoch 1/25
25720/25720 [==============================] - 3s 133us/sample - loss: 0.5663 - accuracy: 0.7205
Epoch 2/25
25720/25720 [==============================] - 2s 86us/sample - loss: 0.5537 - accuracy: 0.7311
Epoch 3/25
25720/25720 [==============================] - 2s 86us/sample - loss: 0.5513 - accuracy: 0.7322
Epoch 4/25
25720/25720 [==============================] - 2s 86us/sample - loss: 0.5496 - accuracy: 0.7326
Epoch 5/25
25720/25720 [==============================] - 2s 87us/sample - loss: 0.5487 - accuracy: 0.7330
Epoch 6/25
25720/25720 [==============================] - 2s 88us/sample - loss: 0.5477 - accuracy: 0.7327
Epoch 7/25
25720/25720 [==============================] - 2s 88us/sample - loss: 0.5464 - accuracy: 0.7339
Epoch 8/25
25720/25720 [==============================] - 2s 89us/sample - loss: 0.5459 - accuracy: 0.7341
Epoch 9/25
25720/25720 [==============================] - 2s 89us/sample - loss: 0.5452 - accuracy: 0.7338
Epoch 10/25
2

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8574/1 - 1s - loss: 0.5217 - accuracy: 0.7299
Loss: 0.5513958321714391, Accuracy: 0.7298810482025146


## Second Attempt

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]

nn_units = nn_input_dim*3

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units, activation='tanh', input_dim=nn_input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/3, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/3, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 132)               5940      
_________________________________________________________________
dense_13 (Dense)             (None, 44)                5852      
_________________________________________________________________
dense_14 (Dense)             (None, 44)                1980      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 45        
Total params: 13,817
Trainable params: 13,817
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Train on 25720 samples
Epoch 1/50
25720/25720 [==============================] - 4s 141us/sample - loss: 0.5680 - accuracy: 0.7222
Epoch 2/50
25720/25720 [==============================] - 3s 101us/sample - loss: 0.5564 - accuracy: 0.7306
Epoch 3/50
25720/25720 [==============================] - 3s 101us/sample - loss: 0.5530 - accuracy: 0.7311
Epoch 4/50
25720/25720 [==============================] - 3s 103us/sample - loss: 0.5515 - accuracy: 0.7308
Epoch 5/50
25720/25720 [==============================] - 3s 103us/sample - loss: 0.5498 - accuracy: 0.7306
Epoch 6/50
25720/25720 [==============================] - 3s 104us/sample - loss: 0.5478 - accuracy: 0.7322
Epoch 7/50
25720/25720 [==============================] - 3s 104us/sample - loss: 0.5469 - accuracy: 0.7333
Epoch 8/50
25720/25720 [==============================] - 3s 104us/sample - loss: 0.5457 - accuracy: 0.7337
Epoch 9/50
25720/25720 [==============================] - 3s 104us/sample - loss: 0.5449 - accuracy: 0.7326
Epoch

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8574/1 - 1s - loss: 0.5219 - accuracy: 0.7274
Loss: 0.554159273474484, Accuracy: 0.7274317741394043


## Third Attempt

## Optimize the Model

In [13]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return nn_model

In [14]:
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2)

In [15]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 54s]
val_accuracy: 0.7181341052055359

Best val_accuracy So Far: 0.7281632423400879
Total elapsed time: 00h 19m 29s
INFO:tensorflow:Oracle triggered exit


In [16]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 5,
 'num_layers': 4,
 'units_0': 3,
 'units_1': 9,
 'units_2': 7,
 'units_3': 7,
 'units_4': 3,
 'units_5': 5,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [17]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8575/1 - 0s - loss: 0.5709 - accuracy: 0.7282
Loss: 0.5547840290931502, Accuracy: 0.7281632423400879


In [18]:
# Save and export results to an HDF5 file
best_model.save('AlphabetSoupCharity_Optimization.h5')